In [33]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import csv

interbankExposure = pd.read_csv('interbankExposures.csv', header=None)
bankEquities = pd.read_csv('bankEquities.csv', header=None)
bank_asset_weighted_network = pd.read_csv('bankAssetWeightedNetwork.csv', header=None)

G = nx.DiGraph(interbankExposure.values)
nnodes = G.number_of_nodes()  # 145
out_degrees = [G.out_degree(n) for n in G.nodes()]
in_degrees = [G.in_degree(n) for n in G.nodes()]

in_Degree = pd.DataFrame(G.in_degree(weight='weight'))
in_Degree = in_Degree.T

out_Degree = pd.DataFrame(G.out_degree(weight='weight'))
out_Degree = out_Degree.T

external_liability = pd.DataFrame(index=range(1), columns=range(145))
external_liability = external_liability.add(in_Degree.loc[1], axis=1)
external_liability = external_liability.sub(out_Degree.loc[1], axis=1)
external_liability = external_liability.sub(bankEquities.loc[0], axis=1)
external_liability = external_liability.T

notdefaulted_external_liability = external_liability[external_liability[:] > 0].dropna(axis=1)
defaulted_external_liability = external_liability[external_liability[:] <= 0].dropna(axis=1)

initial_bank_equities = bankEquities.copy()
shock = 100000
for i in bankEquities:
    bankEquities[i] = bankEquities[i] - shock

notdefaulters = bankEquities[bankEquities.iloc[0:] > 0]
notdefaulters = notdefaulters.dropna(axis=1)
defaulters = bankEquities[bankEquities.iloc[0:] <= 0]
defaulters = defaulters.dropna(axis=1)

notdefaulted_external_asset = bank_asset_weighted_network.reindex(notdefaulters.columns, axis=1)
defaulter_external_assets = bank_asset_weighted_network.reindex(defaulters.columns, axis=1)
ratio = 1 - (defaulter_external_assets.sum(axis=1) / bank_asset_weighted_network.sum(axis=1))

newBankEquity = pd.DataFrame(index=range(1), columns=range(145))
newbanklist = pd.DataFrame(index=range(1), columns=range(145))
defaulteroutDegree = pd.DataFrame(index=range(1), columns=range(145))
notdefaulted_external_asset = pd.DataFrame(index=range(1), columns=range(145))

notdefaulted_external_asset = bank_asset_weighted_network.reindex(notdefaulters.columns, axis=1)
newBankEquity = newBankEquity.sub(notdefaulted_external_liability, fill_value=0)
newBankEquity = newBankEquity.add(notdefaulted_external_asset, fill_value=0)

recovery_rate = 0.5  # 0.5 = 50% recovery rate


In [30]:
def furfine(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, bank_asset_weighted_network, external_liability, in_Degree, out_Degree, recovery_rate):
    notdefaulters_inDegree = in_Degree.reindex(notdefaulters.columns, axis=1)
    notdefaulters_outDegree = out_Degree.reindex(notdefaulters.columns, axis=1)

    for i in defaulters:
        sumDebt = sum(interbankExposure.loc[i, 0:]) * (1 - recovery_rate)
        defaulteroutDegree[i] = sumDebt


    newBankEquity = newBankEquity.sub(defaulteroutDegree, fill_value=0)
    newBankEquity = pd.DataFrame(notdefaulters_inDegree.loc[1] + newBankEquity.loc[0])
    newBankEquity = pd.DataFrame(newBankEquity[0] - notdefaulters_outDegree.loc[1]).T
    newbanklist = newBankEquity[newBankEquity[:] > 0].dropna(axis=1)

    lenDefaulter = len(newBankEquity[newBankEquity[:] <= 0].dropna(axis=1))
    if len(defaulters) == lenDefaulter:
        return notdefaulters
    else:
        newBankEquity = newbanklist
        defaulters = newBankEquity[newBankEquity[:] <= 0].dropna(axis=0)
        notdefaulters = newBankEquity[newBankEquity[:] > 0].dropna(axis=0)
        return furfine(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, bank_asset_weighted_network, external_liability, in_Degree, out_Degree)

In [31]:

x = furfine(defaulters, notdefaulters, newBankEquity, newbanklist, defaulteroutDegree, interbankExposure, bank_asset_weighted_network, external_liability, in_Degree, out_Degree, recovery_rate)

print(x)

#print the number of banks that are not defaulted
print(len(x.columns))

Empty DataFrame
Columns: []
Index: [0]
0


In [22]:

final_bank_equities = pd.DataFrame(index=range(1), columns=range(145))

final_bank_equities.update(x)
final_bank_equities.fillna(0, inplace=True)

bank_losses = initial_bank_equities - final_bank_equities

print("Initial Bank Equities: ")
print(initial_bank_equities)
print("Bank Losses:")
print(bank_losses)


Initial Bank Equities: 
        0       1      2           3         4         5          6    \
0  465710.0  4436.7  13159  16229000.0  438420.0  271300.0  7572800.0   

          7      8         9    ...    135         136       137       138  \
0  14267000.0  99580  178320.0  ...  32812  32123000.0  699090.0  515310.0   

     139         140         141       142       143         144  
0  14259  11382000.0  14033000.0  129430.0  874880.0  26855000.0  

[1 rows x 145 columns]
Bank Losses:
      0       1     2       3       4       5       6       7     8       9    \
0  1000.0  1000.0  1000  1000.0  1000.0  1000.0  1000.0  1000.0  1000  1000.0   

   ...   135     136     137     138   139     140     141     142     143  \
0  ...  1000  1000.0  1000.0  1000.0  1000  1000.0  1000.0  1000.0  1000.0   

      144  
0  1000.0  

[1 rows x 145 columns]


Code :

### Import the necessary libraries

In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

### Load data from the csv files

In [ ]:
interbankExposure = pd.read_csv('interbankExposures.csv', header=None)
bankEquities = pd.read_csv('bankEquities.csv', header=None)
bank_asset_weighted_network = pd.read_csv('bankAssetWeightedNetwork.csv', header=None)


### Create a directed graph from the dataset

In [ ]:
G = nx.DiGraph(interbankExposure.values)

### Calculate the number of nodes in the graph and the in-degree and out-degree of each node

In [4]:
nnodes = G.number_of_nodes()  # 145
out_degrees = [G.out_degree(n) for n in G.nodes()]
in_degrees = [G.in_degree(n) for n in G.nodes()]
print(out_degrees)
print(in_degrees)


[0, 1, 1, 119, 32, 20, 60, 113, 24, 26, 32, 40, 37, 87, 14, 29, 6, 39, 38, 26, 49, 41, 45, 0, 62, 11, 10, 102, 20, 86, 0, 74, 32, 41, 93, 27, 35, 29, 2, 114, 21, 2, 7, 34, 9, 59, 11, 84, 86, 12, 63, 55, 47, 73, 101, 50, 9, 6, 23, 98, 85, 39, 89, 16, 41, 40, 1, 55, 54, 1, 15, 74, 102, 38, 11, 0, 34, 46, 75, 105, 34, 3, 116, 115, 45, 94, 68, 33, 32, 2, 32, 14, 47, 52, 18, 50, 31, 34, 88, 24, 38, 40, 36, 23, 55, 45, 18, 0, 40, 71, 25, 56, 29, 25, 41, 20, 97, 0, 12, 12, 104, 22, 110, 22, 35, 53, 9, 46, 120, 58, 21, 68, 37, 17, 7, 8, 57, 13, 58, 1, 95, 106, 23, 23, 75]
[16, 1, 4, 98, 13, 28, 77, 101, 22, 19, 19, 14, 27, 80, 28, 26, 2, 34, 40, 13, 78, 51, 20, 2, 32, 36, 31, 114, 26, 63, 11, 97, 36, 27, 84, 4, 43, 48, 1, 100, 29, 5, 10, 58, 15, 60, 38, 65, 76, 3, 38, 32, 18, 26, 95, 21, 12, 12, 35, 106, 91, 54, 78, 29, 15, 52, 29, 60, 42, 71, 16, 63, 107, 44, 10, 11, 48, 13, 79, 88, 46, 8, 125, 94, 35, 95, 82, 41, 55, 16, 12, 35, 33, 73, 11, 41, 43, 40, 84, 44, 46, 26, 51, 30, 30, 14, 36, 5, 

### Compute the in degrees for each node (Interbank assets)

In [ ]:
in_Degree = pd.DataFrame(G.in_degree(weight='weight'))
in_Degree = in_Degree.T


### Compute the out degrees for each node (Interbank Liabilities)

In [ ]:
out_Degree = pd.DataFrame(G.out_degree(weight='weight'))
out_Degree = out_Degree.T


### Calculate the external liabilties for each bank

In [ ]:
external_liability = pd.DataFrame(index=range(1), columns=range(145))
external_liability = external_liability.add(in_Degree.loc[1], axis=1)
external_liability = external_liability.sub(out_Degree.loc[1], axis=1)
external_liability = external_liability.sub(bankEquities.loc[0], axis=1)
external_liability = external_liability.T

### Separate banks that are not defaulted and defaulted based on their external liabilities:

In [ ]:
notdefaulted_external_liability = external_liability[external_liability[:] > 0].dropna(axis=1)
defaulted_external_liability = external_liability[external_liability[:] <= 0].dropna(axis=1)


### Apply the shocks

In [ ]:
initial_bank_equities = bankEquities.copy() # Save the initial bank equities to calculate the losses later
shock = 1000000000
for i in bankEquities:
    bankEquities[i] = bankEquities[i] - shock

### Caculate the losses

In [32]:
final_bank_equities = pd.DataFrame(index=range(1), columns=range(145))
final_bank_equities.update(x)
final_bank_equities.fillna(0, inplace=True)
bank_losses = initial_bank_equities - final_bank_equities
print("Bank Losses:")
print(bank_losses)


Bank Losses:
        0       1      2           3         4         5          6    \
0  465710.0  4436.7  13159  16229000.0  438420.0  271300.0  7572800.0   

          7      8         9    ...    135         136       137       138  \
0  14267000.0  99580  178320.0  ...  32812  32123000.0  699090.0  515310.0   

     139         140         141       142       143         144  
0  14259  11382000.0  14033000.0  129430.0  874880.0  26855000.0  

[1 rows x 145 columns]
